In [ ]:
# ==============================
# 운영체제(OS) 관련 기능을 사용하기 위한 기본 라이브러리
# (환경변수 읽기 위해 필요)
# ==============================
import os

# ==============================
# .env 파일을 읽어오기 위한 라이브러리
# .env 파일에는 API KEY 같은 민감한 정보를 저장함
# ==============================
from dotenv import load_dotenv


# ==============================
# .env 파일에 있는 내용을 실제 환경변수로 등록
# ==============================
load_dotenv()


# ==============================
# 운영체제 환경변수에서 OPENAI_API_KEY 값을 읽어옴
# .env 파일 안에 있는 값을 가져오는 역할
# ==============================
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")



In [ ]:
# ==============================
# LangChain에서 제공하는 LLMChain 클래스 불러오기
# → 프롬프트 + GPT 모델을 연결해서 실행하는 체인
# ==============================
from langchain_classic.chains import LLMChain

# ==============================
# PromptTemplate 클래스 불러오기
# → 질문 템플릿(틀)을 만들기 위한 도구
# ==============================
from langchain_classic import PromptTemplate

# ==============================
# ChatGPT(OpenAI Chat 모델)를 사용하기 위한 클래스 불러오기
# ==============================
from langchain_classic.chat_models import ChatOpenAI


# ==============================
# GPT 모델 객체 생성
#
# temperature=0
# → 답변을 랜덤하지 않게, 가장 정확하게 생성
#
# model_name='gpt-4.1-mini'
# → 사용할 GPT 모델 지정 (빠르고 가벼운 모델)
# ==============================
llm=ChatOpenAI(temperature=0,
               model_name='gpt-4.1-mini')


# ==============================
# 프롬프트 템플릿 생성
#
# input_variable=["country"]
# → 프롬프트 안에서 사용할 변수 이름 지정
#
# template
# → 실제 질문 문장 틀
# {country} 부분은 나중에 값이 자동으로 들어감
# ==============================
prompt=PromptTemplate(
    input_variable=["country"],
    template="{country}의 수도는 어디야?",
)


# ==============================
# LLMChain 생성
#
# llm=llm
# → 위에서 만든 GPT 모델 사용
#
# prompt=prompt
# → 위에서 만든 프롬프트 템플릿 사용
#
# 즉:
# "이 프롬프트를 이 GPT 모델로 실행하겠다" 는 의미
# ==============================
chain=LLMChain(llm=llm,prompt=prompt)


# ==============================
# 체인 실행
#
# "대한민국" 값을 PromptTemplate의 {country} 변수에 자동으로 넣음
#
# 내부적으로:
# "{country}의 수도는 어디야?"
# →
# "대한민국의 수도는 어디야?"
#
# 로 변환 후 GPT에게 전달
# ==============================
chain.run("대한민국")

C:\Users\User\AppData\Local\Temp\ipykernel_9592\3816696069.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the `langchain-openai package and should be used instead. To use it run `pip install -U `langchain-openai` and import as `from `langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(temperature=0,
C:\Users\User\AppData\Local\Temp\ipykernel_9592\3816696069.py:13: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  chain=LLMChain(llm=llm,prompt=prompt)
C:\Users\User\AppData\Local\Temp\ipykernel_9592\3816696069.py:14: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  chain.run("대한민국")


'대한민국의 수도는 서울입니다.'

In [ ]:
# ==============================
# 1️⃣ 번역용 프롬프트 템플릿 생성
# ==============================
prompt1=PromptTemplate(
     # 이 프롬프트에서 사용할 입력 변수 이름
    # {sentence} 자리에 실제 문장이 들어감
    input_variables=['sentence'],
    # GPT에게 전달될 질문 템플릿
    # \n\n 은 줄바꿈 (프롬프트 가독성 증가)
    template="다음 문장을 한글로 번역하세요 .\n\n{sentence}"

)

# ==============================
# 2️⃣ 첫 번째 체인 생성 (번역 체인)
# ==============================
chain1=LLMChain(llm=llm,# 사용할 GPT 모델
                prompt=prompt1,# 위에서 만든 번역 프롬프트 사용

                # 이 체인의 출력 결과 이름 지정
                # 결과가 "translation" 이라는 키로 저장됨
                output_key='translation')


# ==============================
# 3️⃣ 요약용 프롬프트 템플릿 생성
# ==============================
prompt2=PromptTemplate.from_template(
     # {translation} 은
    # chain1 의 output_key 값과 자동 연결됨
    "다음 문장을 한 문장으로 요약하세요 .\n\n{translation}"
)

# ==============================
# 4️⃣ 두 번째 체인 생성 (요약 체인)
# ==============================
chain2=LLMChain(llm=llm,# 동일한 GPT 모델 사용
                prompt=prompt2, # 요약 프롬프트 사용
                 # 이 체인의 출력 결과 이름
                output_key='summary')

from langchain_classic.chains import SequentialChain

# ==============================
# 6️⃣ 전체 파이프라인 체인 구성
# ==============================
all_chain=SequentialChain(
     # 실행할 체인 순서 지정
    # → 먼저 chain1 실행
    # → 그 다음 chain2 실행
    chains=[chain1,chain2],
      # 사용자로부터 처음 입력 받을 변수 이름
    # 즉 all_chain() 실행 시 sentence 값을 받아야 함
    input_variables=['sentence'],
    # 최종 출력으로 받을 결과들
    # chain1 결과: translation
    # chain2 결과: summary
    output_variables=['translation','summary'],
)

# ==============================
# 7️⃣ 입력 문장 (영어 원문)
# ==============================
sentence="""
One limitation of LLMs is their lack of contextual information (e.g., access to some specific documents or emails). You can combat this by giving LLMs access to the specific external data.
For this, you first need to load the external data with a document loader. LangChain provides a variety of loaders for different types of documents ranging from PDFs and emails to websites and YouTube videos.
"""


# ==============================
# 8️⃣ 전체 체인 실행
# ==============================

# all_chain(sentence) 실행 시:
#
# 1) sentence 값이 chain1으로 전달됨
# 2) chain1이 번역 수행 → translation 생성
# 3) translation 결과가 chain2로 자동 전달
# 4) chain2가 요약 수행 → summary 생성
# 5) translation + summary 결과 반환
all_chain(sentence)

C:\Users\User\AppData\Local\Temp\ipykernel_9592\3898507903.py:28: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  all_chain(sentence)


{'sentence': '\nOne limitation of LLMs is their lack of contextual information (e.g., access to some specific documents or emails). You can combat this by giving LLMs access to the specific external data.\nFor this, you first need to load the external data with a document loader. LangChain provides a variety of loaders for different types of documents ranging from PDFs and emails to websites and YouTube videos.\n',
 'translation': 'LLM의 한 가지 한계는 특정 문서나 이메일과 같은 맥락 정보를 갖고 있지 않다는 점입니다. 이를 해결하려면 LLM이 특정 외부 데이터에 접근할 수 있도록 해야 합니다.  \n이를 위해 먼저 문서 로더를 사용해 외부 데이터를 불러와야 합니다. LangChain은 PDF, 이메일부터 웹사이트, 유튜브 동영상에 이르기까지 다양한 유형의 문서를 위한 여러 로더를 제공합니다.',
 'summary': 'LLM의 한계를 극복하기 위해 LangChain의 다양한 문서 로더를 활용해 PDF, 이메일, 웹사이트, 유튜브 동영상 등 외부 데이터를 불러와 접근할 수 있도록 해야 합니다.'}